In [276]:
using JuMP
using GLPK

In [277]:
AVAIL = Dict("crude2" => 500000, "crude1" => 250000)

DEM = Dict(
  "butane"  => 20000,
  "heating" => 42000,
  "petrol"  => 40000,
  "diesel"  => 30000,
    )

DIST = Dict(
    ("crude1", "gasoil")     => 0.4, 
    ("crude1", "residue")    => 0.15,
    ("crude1", "naphtha")    => 0.15,
    ("crude2", "gasoil")     => 0.35,
    ("crude2", "residue")    => 0.1,
    ("crude1", "distbutane") => 0.03,
    ("crude2", "naphtha")    => 0.2,
    ("crude2", "distbutane") => 0.05
    )

REF = Dict(
    "reformate" => 0.85,
    "refbutane" => 0.15
    )

CRACK = Dict(
    "crknaphtha" => 0.4,
    "crkgasoil"  => 0.35
    )

OCT = Dict(
    "petcrknaphtha" => 74,
    "reformate"     => 100,
    "petbutane"     => 120
    )

VAP = Dict("petbutane" =>  60, "reformate" => 2.6, "petcrknaphtha" => 4.1)
VOL = Dict("petbutane" => 105, "reformate" => 3, "petcrknaphtha" => 12)

SULF = Dict("dslgasoil" => 0.03, "dslcrknaphtha" => 0.12, "dslcrkgasoil" => 0.76)

COST = Dict("crude1" => 2.1, 
    "crude2" => 2.1,
    "distbutane" => 0,
    "naphtha" => 4.18,
    "residue" => 0.6,
    "gasoil" => 2.04
    )

Dict{String,Real} with 6 entries:
  "naphtha"    => 4.18
  "residue"    => 0.6
  "distbutane" => 0
  "crude2"     => 2.1
  "crude1"     => 2.1
  "gasoil"     => 2.04

In [278]:
CRUDES = collect(keys(AVAIL))
FINAL = collect(keys(DEM))
IDIST = unique([x[2] for x in collect(keys(DIST))])
IREF = collect(keys(REF))
ICRACK = collect(keys(CRACK))
IPETROL = collect(keys(OCT))
IDIESEL = collect(keys(SULF))
IHO = ["hogasoil", "hocrknaphtha", "hocrkgasoil"]

3-element Array{String,1}:
 "hogasoil"
 "hocrknaphtha"
 "hocrkgasoil"

In [279]:
ALLPRODS = unique([FINAL; IDIST; IREF; ICRACK; IPETROL; IHO; IDIESEL])

20-element Array{String,1}:
 "butane"
 "heating"
 "petrol"
 "diesel"
 "gasoil"
 "residue"
 "naphtha"
 "distbutane"
 "reformate"
 "refbutane"
 "crknaphtha"
 "crkgasoil"
 "petcrknaphtha"
 "petbutane"
 "hogasoil"
 "hocrknaphtha"
 "hocrkgasoil"
 "dslcrknaphtha"
 "dslgasoil"
 "dslcrkgasoil"

In [280]:
model = Model(GLPK.Optimizer)
@variable(model, c[1:2] >= 0)
set_name.(c, CRUDES)

@variable(model, x[1:length(ALLPRODS)] >= 0)
set_name.(x, ALLPRODS);

# Objective function
@objective(model, Min, sum(variable_by_name(model, i) * COST[i] for i in CRUDES) + sum(variable_by_name(model, i) * COST[i] for i in IDIST))

# Relations intermediate products resulting of distillation - raw materials
for p in IDIST
    @constraint(model, variable_by_name(model, p) <= sum(DIST[(i, p)] * variable_by_name(model, i) for i in CRUDES))
end

# Relations between intermediate products
# Reforming:
for p in IREF
    @constraint(model, variable_by_name(model, p) <= REF[p] * variable_by_name(model, "naphtha"))
end

#Cracking:
for p in ICRACK
    @constraint(model, variable_by_name(model, p) <= CRACK[p] * variable_by_name(model, "residue"))
end

@constraint(model, variable_by_name(model, "crknaphtha") == variable_by_name(model, "petcrknaphtha") +
    variable_by_name(model, "hocrknaphtha") +
    variable_by_name(model, "dslcrknaphtha")
    )
@constraint(model, variable_by_name(model, "crkgasoil") == variable_by_name(model, "hocrkgasoil") +
    variable_by_name(model, "dslcrkgasoil")
    )

# Desulfurization:
@constraint(model, variable_by_name(model, "gasoil") == variable_by_name(model, "hogasoil") +
    variable_by_name(model, "dslgasoil")
    )

# Relations final products - intermediate products
@constraint(model, variable_by_name(model, "butane") == variable_by_name(model, "distbutane") +
    variable_by_name(model, "refbutane") -
    variable_by_name(model, "petbutane")
    )

@constraint(model, variable_by_name(model, "petrol") == sum(variable_by_name(model, p) for p in IPETROL))
@constraint(model, variable_by_name(model, "diesel") == sum(variable_by_name(model, p) for p in IDIESEL))
@constraint(model, variable_by_name(model, "heating") == sum(variable_by_name(model, p) for p in IHO))

# Properties of petrol
@constraint(model, sum(OCT[p] * variable_by_name(model, p) for p in IPETROL) >= 94 * variable_by_name(model, "petrol"))
@constraint(model, sum(VAP[p] * variable_by_name(model, p) for p in IPETROL) <= 12.7 * variable_by_name(model, "petrol"))
@constraint(model, sum(VOL[p] * variable_by_name(model, p) for p in IPETROL) >= 17 * variable_by_name(model, "petrol"))

# Limit on sulfur in diesel oil
@constraint(model, sum(SULF[p] * variable_by_name(model, p) for p in IDIESEL) <= 0.05 * variable_by_name(model, "diesel"))

# Crude availabilities
for crud in CRUDES
    @constraint(model, variable_by_name(model, crud) <= AVAIL[crud])
end


# Production capacities
@constraint(model, variable_by_name(model, "naphtha") <= 30000)               # Reformer
@constraint(model, variable_by_name(model, "gasoil") <= 50000)                # Desulfurization
@constraint(model, variable_by_name(model, "residue") <= 40000)               # Cracker

# Satisfy demands
for p in FINAL
   @constraint(model, variable_by_name(model, p) >= DEM[p])
end

In [281]:
println(model)

Min 2.1 crude2 + 2.1 crude1 + 2.04 gasoil + 0.6 residue + 4.18 naphtha
Subject to
 crknaphtha - petcrknaphtha - hocrknaphtha - dslcrknaphtha == 0.0
 crkgasoil - hocrkgasoil - dslcrkgasoil == 0.0
 gasoil - hogasoil - dslgasoil == 0.0
 butane - distbutane - refbutane + petbutane == 0.0
 petrol - petcrknaphtha - reformate - petbutane == 0.0
 diesel - dslcrknaphtha - dslgasoil - dslcrkgasoil == 0.0
 heating - hogasoil - hocrknaphtha - hocrkgasoil == 0.0
 74 petcrknaphtha + 100 reformate + 120 petbutane - 94 petrol >= 0.0
 12 petcrknaphtha + 3 reformate + 105 petbutane - 17 petrol >= 0.0
 butane >= 20000.0
 heating >= 42000.0
 petrol >= 40000.0
 diesel >= 30000.0
 gasoil - 0.35 crude2 - 0.4 crude1 <= 0.0
 residue - 0.1 crude2 - 0.15 crude1 <= 0.0
 naphtha - 0.2 crude2 - 0.15 crude1 <= 0.0
 distbutane - 0.05 crude2 - 0.03 crude1 <= 0.0
 reformate - 0.85 naphtha <= 0.0
 refbutane - 0.15 naphtha <= 0.0
 crknaphtha - 0.4 residue <= 0.0
 crkgasoil - 0.35 residue <= 0.0
 4.1 petcrknaphtha + 2.6 r

In [282]:
optimize!(model)

In [283]:
JuMP.termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [285]:
JuMP.getobjectivevalue(model)

1.1754e6

In [286]:
hcat(ALLPRODS, value.(x))

20×2 Array{Any,2}:
 "butane"         20000.0
 "heating"        42000.0
 "petrol"         40000.0
 "diesel"         30000.0
 "gasoil"         50000.0
 "residue"        40000.0
 "naphtha"        30000.0
 "distbutane"     22000.0
 "reformate"      25500.0
 "refbutane"       4500.0
 "crknaphtha"     16000.0
 "crkgasoil"      14000.0
 "petcrknaphtha"   8000.0
 "petbutane"       6500.0
 "hogasoil"       26666.7
 "hocrknaphtha"    1333.33
 "hocrkgasoil"    14000.0
 "dslcrknaphtha"   6666.67
 "dslgasoil"      23333.3
 "dslcrkgasoil"       0.0